In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/healthcare/healthcare_dataset.csv


# Isolation Forest Anomaly Detection HealthCare

In [2]:

import matplotlib.pyplot as plt


df = pd.read_csv('/kaggle/input/healthcare/healthcare_dataset.csv')

In [3]:
df

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55495,eLIZABeTH jaCkSOn,42,Female,O+,Asthma,2020-08-16,Joshua Jarvis,Jones-Thompson,Blue Cross,2650.714952,417,Elective,2020-09-15,Penicillin,Abnormal
55496,KYle pEREz,61,Female,AB-,Obesity,2020-01-23,Taylor Sullivan,Tucker-Moyer,Cigna,31457.797307,316,Elective,2020-02-01,Aspirin,Normal
55497,HEATher WaNG,38,Female,B+,Hypertension,2020-07-13,Joe Jacobs DVM,"and Mahoney Johnson Vasquez,",UnitedHealthcare,27620.764717,347,Urgent,2020-08-10,Ibuprofen,Abnormal
55498,JENniFER JOneS,43,Male,O-,Arthritis,2019-05-25,Kimberly Curry,"Jackson Todd and Castro,",Medicare,32451.092358,321,Elective,2019-05-31,Ibuprofen,Abnormal


In [4]:
df.columns

Index(['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition',
       'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider',
       'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date',
       'Medication', 'Test Results'],
      dtype='object')

# Removing Unnecessary Columns

In [5]:
df = df.drop(["Date of Admission"], axis=1)




In [6]:
df = df.drop(["Discharge Date"], axis=1)


In [7]:
df = df.drop(['Doctor'], axis=1)



In [8]:
df = df.drop(['Hospital'], axis=1)


In [9]:
df = df.drop(['Room Number'], axis=1)


In [10]:
df = df.drop(['Name'], axis=1)

In [11]:
df = df.drop(['Insurance Provider'], axis=1)

In [12]:
df = df.drop(['Billing Amount'], axis=1)

# Label Encoding

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
lc=LabelEncoder()
cols=[ 'Gender','Blood Type', 'Medical Condition','Admission Type', 'Medication','Test Results']
for i in cols:
  df[i]=lc.fit_transform(df[i])

In [15]:
df.isna().sum()

Age                  0
Gender               0
Blood Type           0
Medical Condition    0
Admission Type       0
Medication           0
Test Results         0
dtype: int64

In [16]:
y = df['Test Results']
x = df.drop(['Test Results'], axis=1)
x


,Age,Gender,Blood Type,Medical Condition,Admission Type,Medication
0,30,1,5,2,2,3
1,62,1,0,5,1,1
2,76,0,1,5,1,0
3,28,0,6,3,0,1
4,43,0,2,2,2,4
...,...,...,...,...,...,...
55495,42,0,6,1,0,4
55496,61,0,3,5,0,0
55497,38,0,4,4,2,1
55498,43,1,7,0,0,1


In [17]:
y

0        2
1        1
2        2
3        0
4        0
        ..
55495    0
55496    2
55497    0
55498    0
55499    0
Name: Test Results, Length: 55500, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=42)



In [19]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

y_pred


array([0, 2, 1, ..., 2, 1, 0])

In [20]:

from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)
accuracy

0.3557193557193557